# Calibrating the global orientation of the TART

Author: Tim Molteno tim@elec.ac.nz

We use google maps to do this and find an arm that aligns closely to a distant landmark.

In [1]:
!pip install numpy pyproj tart-tools
import numpy as np


/bin/bash: line 1: pip: command not found


In [2]:
import pyproj
import numpy as np

geodesic = pyproj.Geod(ellps='WGS84')

def bearing(lat1,  lon1,  lat2, lon2):
    fwd_azimuth,back_azimuth,distance = geodesic.inv(lon1, lat1, lon2, lat2)
    return fwd_azimuth

def bearing_latlon(a, b):
    return bearing(lon1=a[1], lat1=a[0], lon2=b[1], lat2=b[0])

def get_rotation_angle(expected_angle, actual_angle):
    print(f"expected angle {expected_angle}")
    print(f"actual angle {actual_angle}")
    rotation =  -(actual_angle - expected_angle)
    return rotation

def get_geographic_angle(dx, dy):
    ''' Diven an x (east is +x) and y (north is +y) delta, 
        return the angle from north.
        
        positive angles point clockwise from geographic north.

    '''
    brng = np.arctan2(dx,dy) # count degrees counter-clockwise
    brng = np.degrees(brng)
    brng = (brng + 360) % 360;
    return brng; 

In [3]:
get_geographic_angle(1, 0)

np.float64(90.0)

In [4]:
import json
from tart.operation import settings

def create_calibrated_positions(original_positions, fname, _rot):
    " Rotate the positions and write to file."    
    print(f"Rotating positions by {_rot} degrees")
    _rot_rad = np.radians(_rot)
                
    new_positions = settings.rotate_location(
                    np.degrees(_rot_rad), np.array(original_positions).T)
                
    _pos_list = (np.array(new_positions).T).tolist()

    with open(fname, 'w') as f:
        json.dump(_pos_list, f, indent=4)
    print(f"Written file '{fname}' with calibrated positions")


ModuleNotFoundError: No module named 'tart'

# Rhodes Site

Measurement was taken along arm 5 that pointed to a distant wind turbine.

In [ ]:
# Rhodes Site
tree = [-33.314532, 26.504503];
monument = [-33.319285, 26.519030]
turbine = [-33.326303, 26.469101] 
rhodes_tart = [-33.319449, 26.507807]; 

ang_tart_turbine = bearing_latlon(rhodes_tart, turbine)

# positive angles point west of geographic north. (clockwise)
ang_tart_turbine

In [ ]:
expected_angle =  -72  # Arm 5
actual_angle = ang_tart_turbine
get_rotation_angle(expected_angle, ang_tart_turbine)

In [ ]:
# Import unaligned positions from json

with open('rhodes/rhodes_antenna_positions_nonaligned.json') as f:
    nonaligned_json = json.load(f)
    ant_pos_unaligned = nonaligned_json['ant_pos']

for i, p in enumerate(ant_pos_unaligned):
    print(i,p)

In [ ]:
import matplotlib.pyplot as plt

for i, p in enumerate(ant_pos_unaligned):
    plt.plot(p[0], p[1], '*')
plt.show()

In [ ]:
ant20 = ant_pos_unaligned[20]
x,y,z = ant20

expected_angle = get_geographic_angle(x,y)
expected_angle

In [ ]:
rot = get_rotation_angle(expected_angle, ang_tart_turbine)
rot

Now enter this information into the global_rotation field in the site survey notebook

## UdM site in Mauritius

The bearing was taken along the path to antenna 4 which featured a power pole in the distance

In [ ]:
# UdM site
udm_tart = [-20.2587508, 57.75919894]
udm_pole = [-20.257990, 57.760779]

ang_tart_pole = bearing_latlon(udm_tart, udm_pole)

ang_tart_pole

In [ ]:
# This is with antenna 4
ant4 = [1.6261960453222395, 0.7033414756277336, 0.0]
x,y,z = ant4

expected_angle = get_geographic_angle(x,y)
expected_angle

In [ ]:
get_rotation_angle(expected_angle, ang_tart_pole)

In [ ]:
# Import unaligned positions from json

with open('mu-udm/mu-udm_antenna_positions_unaligned.json') as f:
    nonaligned_json = json.load(f)
    ant_pos_unaligned = nonaligned_json['antenna_positions']

for i, p in enumerate(ant_pos_unaligned):
    print(i,p)

ant4 = ant_pos_unaligned[4]
x,y,z = ant4

expected_angle = get_geographic_angle(x,y)
expected_angle

In [ ]:
rot = get_rotation_angle(expected_angle, ang_tart_pole)
create_calibrated_positions(ant_pos_unaligned, 
                            fname = 'mu-udm/calibrated_antenna_positions.json',
                            _rot=rot)

## Nairobi site in Kenya

The bearing was taken along the path to antenna 4 which featured a distant highrise building

In [ ]:
from IPython.display import display, Image
display(Image(filename='kenya/kenya_align.jpg'))

In [ ]:
kenya_tart = [-1.2913157,36.8193813] 
kenya_bldg = [-1.292174, 36.825535]

ang_tart_bldg = bearing_latlon(kenya_tart, kenya_bldg)

ang_tart_bldg


In [ ]:
# Import unaligned positions from json

with open('kenya/kenya_antenna_positions_nonaligned.json') as f:
    nonaligned_json = json.load(f)
    ant_pos_unaligned = nonaligned_json['ant_pos']

In [ ]:
for i, p in enumerate(ant_pos_unaligned):
    print(i,p)

In [ ]:
# Use antenna #5 (index=4)

ant5 = ant_pos_unaligned[4]
x,y,z = ant5

expected_angle = get_geographic_angle(x,y)
expected_angle

In [ ]:
rot = get_rotation_angle(expected_angle, ang_tart_bldg)
rot

In [ ]:
create_calibrated_positions(ant_pos_unaligned, fname = 'kenya/calibrated_antenna_positions.json')

In [ ]:
!tart_upload_antenna_positions --api=https://api.elec.ac.nz/tart/tart-kenya --file kenya/antenna_positions_kenya.json --pw sharkbait

## Ghana TART



In [ ]:
display(Image(filename='ghana/antena_9_dish.jpg'))

In [ ]:
ghana_tart = [5.749725,-0.3055573] 
ghana_dish = [5.750541, -0.304980]

ang_ghana_dish = bearing_latlon(ghana_tart, ghana_dish)

ang_ghana_dish
